In [32]:
import pandas as pd
import numpy as np
import datetime

In [33]:
hoy = datetime.datetime.today()

In [34]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd

# Inicializar tkinter y ocultar la ventana principal
root = tk.Tk()
root.withdraw() 

# Abrir el explorador de archivos para seleccionar un archivo
filepath = filedialog.askopenfilename(
    title="Selecciona un archivo",
    filetypes=(("Todos los archivos", "*.*"),("Archivos de texto", "*.txt"))
)

# Imprimir la ruta del archivo seleccionado
if filepath:  # Asegurarse de que el usuario no canceló la selección
    print(f"Ruta del archivo seleccionado: {filepath}")

    # Ejemplo de cómo usar la ruta para leer un archivo CSV
    try:
        ruta = filepath
        print("DataFrame cargado exitosamente:")
    
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
else:
    print("No se seleccionó ningún archivo.")

Ruta del archivo seleccionado: C:/Users/lravlic/codigos/dispo_bmw/transito_raw.xlsx
DataFrame cargado exitosamente:


In [35]:
tr = pd.read_excel(ruta)

In [36]:
#crear lt fact y total
condicion = [tr['Tipo.pedido']=='S',tr['Tipo.pedido']=='U',tr['Tipo.pedido']=='V']
resultado = [30,10,5]
resultado_2 = [90,25,15]

tr['LT Fact'] = np.select(condicion,resultado,0)
tr['LT Total'] = np.select(condicion,resultado_2,0)



In [37]:
tr['Fecha.creación'] = pd.to_datetime(tr['Fecha.creación'])

In [38]:
tr['Fecha O'] = np.where(
    tr['Status.línea.pedido'] == 'O',
    np.where(
        tr['Fecha.creación'] + pd.to_timedelta(tr['LT Fact'], unit='D') >= pd.to_datetime(hoy),
        tr['Fecha.creación'] + pd.to_timedelta(tr['LT Total'], unit='D'),
        pd.to_datetime(hoy) + pd.to_timedelta(tr['LT Total'] * 1.5, unit='D')
    ),
    pd.NaT  # Reemplaza 0 con un valor de fecha/tiempo nulo
)

In [39]:
tr['Fecha O'] = pd.to_datetime(tr['Fecha O'])

In [40]:
# Inicializar tkinter y ocultar la ventana principal
root = tk.Tk()
root.withdraw() 

# Abrir el explorador de archivos para seleccionar un archivo
filepath = filedialog.askopenfilename(
    title="Selecciona un archivo",
    filetypes=(("Todos los archivos", "*.*"),("Archivos de texto", "*.txt"))
)

# Imprimir la ruta del archivo seleccionado
if filepath:  # Asegurarse de que el usuario no canceló la selección
    print(f"Ruta del archivo seleccionado: {filepath}")

    # Ejemplo de cómo usar la ruta para leer un archivo CSV
    try:
        ruta = filepath
        print("DataFrame cargado exitosamente:")
    
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
else:
    print("No se seleccionó ningún archivo.")

Ruta del archivo seleccionado: C:/Users/lravlic/codigos/dispo_bmw/etas.xlsx
DataFrame cargado exitosamente:


In [41]:
etas = pd.read_excel(ruta)

In [42]:
# 1. Convert all date-like columns to datetime format.
#    The `errors='coerce'` will turn any invalid values (like '0') into NaT.
etas['Fecha Arribo Bodega'] = pd.to_datetime(etas['Fecha Arribo Bodega'], errors='coerce')
etas['Retiro puerto'] = pd.to_datetime(etas['Retiro puerto'], errors='coerce')
etas['Manifiesto'] = pd.to_datetime(etas['Manifiesto'], errors='coerce')
etas['ETA inicial'] = pd.to_datetime(etas['ETA inicial'], errors='coerce')

# 2. Apply the selection logic using .notna()
#    This is much cleaner and more reliable than comparing to 0.
etas['Fecha consolidada'] = np.where(
    etas['Fecha Arribo Bodega'].notna(),
    etas['Fecha Arribo Bodega'],
    np.where(
        etas['Retiro puerto'].notna(),
        etas['Retiro puerto'],
        np.where(
            etas['Manifiesto'].notna(),
            etas['Manifiesto'],
            etas['ETA inicial']
        )
    )
)

In [43]:
tr['Nº.de.referencia'] = tr['Nº.de.referencia'].astype('str')

In [44]:


tr['Factura'] = np.where(
    tr['Status.línea.pedido'] == 'A',
    '872' + tr['Nº.de.referencia'],
    "0"
)

In [45]:
etas.drop_duplicates(subset='Factura BMW', inplace=True)

In [46]:
tr['Factura'] = tr['Factura'].astype('str')
etas['Factura BMW'] = etas['Factura BMW'].astype('str')

In [47]:
tr_etas = tr.merge(etas[['Factura BMW','Fecha consolidada']], left_on='Factura', right_on='Factura BMW', how='left')

In [48]:
tr_etas.drop(columns=['Factura BMW'], inplace=True)

In [49]:
tr_etas['Fecha A'] = np.where(
    tr_etas['Status.línea.pedido']=='A',
    tr_etas['Fecha consolidada'],
    pd.NaT
    
)

In [50]:
tr_etas['Fecha A'] = pd.to_datetime(tr_etas['Fecha A'])

In [51]:
tr_etas['Status'] = np.where(
    tr_etas['Status.línea.pedido'] == 'O',
    np.where(
        tr['Fecha.creación'] + pd.to_timedelta(tr['LT Fact'], unit='D') >= pd.to_datetime(hoy),"Transito","Transito Vencido"
    ),
    np.where(tr_etas['Status.línea.pedido']=="A",
             np.where(
                  tr_etas['Fecha A']  >= pd.to_datetime(hoy),"Facturado", "Facturado Vencido"
             ),"0") 
)

In [52]:
tr_etas['Status'].value_counts()

Status
0                    40200
Facturado Vencido     1348
Facturado              960
Transito Vencido       702
Name: count, dtype: int64

In [ ]:
# Define the conditions and corresponding choices
conditions = [
    (tr_etas['Status.línea.pedido'] == 'A') & (tr_etas['Tipo.pedido'] == 'S') & (tr_etas['Status'] == 'Facturado Vencido'),
    (tr_etas['Status.línea.pedido'] == 'A') & (tr_etas['Tipo.pedido'] == 'U') & (tr_etas['Status'] == 'Facturado Vencido'),
    (tr_etas['Status.línea.pedido'] == 'A') & (tr_etas['Tipo.pedido'] == 'V') & (tr_etas['Status'] == 'Facturado Vencido'),
    (tr_etas['Status.línea.pedido'] == 'A'),
    (tr_etas['Status.línea.pedido'] == 'O')
]

# Define the choices that correspond to each condition
choices = [
    pd.to_datetime('today').normalize() + pd.Timedelta(days=45),
    pd.to_datetime('today').normalize() + pd.Timedelta(days=15),
    pd.to_datetime('today').normalize() + pd.Timedelta(days=5),
    tr_etas['Fecha A'],
    tr_etas['Fecha O']
]

# Apply the conditions using np.select()
tr_etas['Fecha Final'] = np.select(conditions, choices, default=0)

In [ ]:
tr_etas['Fecha Final'] = pd.to_datetime(tr_etas['Fecha Final'])

In [ ]:
tr_etas['Fecha Final'].value_counts()

Resultado
1970-01-01 00:00:00.000000    40200
2025-11-13 00:00:00.000000      988
2025-10-01 00:00:00.000000      822
2025-11-05 22:06:13.719778      465
2025-10-14 00:00:00.000000      338
2026-02-11 10:06:13.719778      221
2025-10-15 00:00:00.000000      138
2025-10-04 00:00:00.000000       22
2025-10-21 22:06:13.719778       15
2025-09-29 10:06:13.719778        1
Name: count, dtype: int64

In [57]:
condicion = [tr_etas['Status.línea.pedido']=='A', tr_etas['Status.línea.pedido']=='O']
opcion = [tr_etas['Cantidad.aconsejada'], tr_etas['Cantidad.requerida']]

tr_etas['Cantidad Final'] = np.select(condicion, opcion, 0)

In [58]:
# Inicializar tkinter y ocultar la ventana principal
root = tk.Tk()
root.withdraw() 

# Abrir el explorador de archivos para guardar el DataFrame
filepath = filedialog.asksaveasfilename(
    defaultextension=".xlsx",
    filetypes=( ("Archivos de Excel", "*.xlsx"),("Archivos XLSX", "*.xlsx")),
    title="Guardar DataFrame como..."
)

# Exportar el DataFrame si el usuario seleccionó una ubicación
if filepath:
    try:
        # Exportar a CSV
        if filepath.endswith('.csv'):
            tr_etas.to_csv(filepath, index=False)
            print(f"DataFrame exportado exitosamente a: {filepath}")
        
        # Exportar a Excel (requiere la librería openpyxl)
        elif filepath.endswith('.xlsx'):
            tr_etas.to_excel(filepath, index=False, engine='openpyxl')
            print(f"DataFrame exportado exitosamente a: {filepath}")
        
        else:
            print("Extensión de archivo no soportada.")

    except Exception as e:
        print(f"Error al exportar el archivo: {e}")
else:
    print("No se seleccionó una ruta de guardado.")

DataFrame exportado exitosamente a: C:/Users/lravlic/codigos/dispo_bmw/transito_bmw.xlsx
